In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [4]:
# 1. Chuẩn bị dữ liệu
mnist = fetch_openml('mnist_784')
X, y = mnist['data'], mnist['target']
X = X / 255.0  # Chuẩn hóa dữ liệu
y = y.astype(int).to_numpy()

# One-hot encoding cho nhãn
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [5]:
# 2. Xây dựng mô hình mạng nơ-ron
input_size = 784  # 28x28 pixels
hidden_size = 64  # Số lượng nơ-ron trong lớp ẩn
output_size = 10  # 10 chữ số (0-9)

In [6]:
# 3. Khởi tạo trọng số và độ lệch
def init_weights(input_size, hidden_size, output_size):
    W1 = np.random.randn(input_size, hidden_size) * 0.01
    b1 = np.zeros((1, hidden_size))
    W2 = np.random.randn(hidden_size, output_size) * 0.01
    b2 = np.zeros((1, output_size))
    return W1, b1, W2, b2

W1, b1, W2, b2 = init_weights(input_size, hidden_size, output_size)

In [7]:
# 4. Triển khai lan truyền tiến
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def forward_propagation(X, W1, b1, W2, b2):
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)
    return A1, A2

In [8]:
# 5. Tính toán hàm mất mát
def compute_loss(A2, Y):
    m = Y.shape[0]
    log_likelihood = -np.log(A2[range(m), np.argmax(Y, axis=1)])
    loss = np.sum(log_likelihood) / m
    return loss

In [9]:
# 6. Triển khai lan truyền ngược
def backward_propagation(X, Y, A1, A2, W2):
    m = X.shape[0]
    dZ2 = A2 - Y
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m
    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * A1 * (1 - A1)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m
    return dW1, db1, dW2, db2


In [10]:
# 7. Huấn luyện mô hình
learning_rate = 0.1
num_epochs = 1000

for epoch in range(num_epochs):
    A1, A2 = forward_propagation(X_train, W1, b1, W2, b2)
    loss = compute_loss(A2, y_train)
    dW1, db1, dW2, db2 = backward_propagation(X_train, y_train, A1, A2, W2)

    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss}')

Epoch 0, Loss: 2.3035017650148224
Epoch 100, Loss: 2.2816560733050975
Epoch 200, Loss: 2.123573463336835
Epoch 300, Loss: 1.6499228707974096
Epoch 400, Loss: 1.2266337027866527
Epoch 500, Loss: 0.9656375357011866
Epoch 600, Loss: 0.7985293829543312
Epoch 700, Loss: 0.6859027716443592
Epoch 800, Loss: 0.6078044878346561
Epoch 900, Loss: 0.5513681455635602


In [1]:
def test_prediction(index, weights):
    #Lấy mẫu dữ liệu từ X_train tại chỉ số index. Sử dụng None để thêm một chiều mới, biến current_image thành mảng 2D với kích thước (1, số_lượng_đặc_trưng)
    current_image = X_train[None, index, :]
    "YOUR CODE HERE"
    #Làm tròn giá trị dự đoán
    prediction = np.round(predict(pad(current_image), weights, threshold=0.5)).astype(int)
    label = y_train[index]

    print("Prediction: ", prediction)
    print("Label: ", label)

    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [11]:
# Đánh giá mô hình trên tập kiểm tra
_, A2_test = forward_propagation(X_test, W1, b1, W2, b2)
predictions = np.argmax(A2_test, axis=1)
labels = np.argmax(y_test, axis=1)
accuracy = np.mean(predictions == labels)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 87.09%
